In [1]:
%matplotlib inline

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

#torchvision을 사용해 cifar 10을 불러오는 과정
#import torch : 신경망 구축 및 훈련을 위해 pytorch 라이브러리를 가져옴
#import torchvision : pytorch 패키지인 torchvision 라이브러리를 가져옴 
# -> mnist, cifar-10 등 인기있는 데이터 셋 등 사전 훈련된 모델이 포함됨
#import torchvision.transforms as transforms -> 입력 데이터에 적용할 수 있는
#공통 이미지 변환 셋을 제공하는 torchvision에서 변환 모듈을 가져온다;
#크기조정, 자르기, 뒤집기 및 정규화와 같은 작업이 포함되며 신경망 훈련 또는
#테스트를 위해 데이터를 준비하는데 사용할 수 있음

#torchvision 데이터셋의 출력은 [0,1] 범위를 갖는 이미지 -> [-1,1]의 범위로 정규화된 tensor로 변환

In [3]:
transform = transforms.Compose(
[transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#transforms.ToTensor() -> 입력 데이터를 신경망에서 처리할 수 있는 pytorch tensor로 변환
#[0,255] 범위에서 [0,1] 범위까지 픽셀 값의 크기를 조정
#transforms.Normalize() -> 평균(0.5,0.5,0.5)를 빼고 표준 편차(0.5,0.5,0.5)로 나누어
#입력 데이터를 정규화
#입력 데이터의 평균 및 단위 분산이 0이 되어 신경망의 성능과 안정성을 개선하는데 도움이 됨

#두개의 테스트 셋에 동일한 변환 파이프라인을 적용해 입력 데이터가 일관된 속성을 갖도록 보장

In [ ]:
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train = True, download = True, transform = transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True,num_workers = 2)

testset = torchvision.datasets.CIFAR10(root='./data', train = False, download = True, transform = transform)

testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 0)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#배치 크기를 4로 정의 : 각 반복에서 로드되고 처리될 이미지 수
#cifar10의 데이터 셋을 다운로드 하고 두 개의 데이터 셋(trainset / testset)을 생성
#trainset은 모델 학습에 사용, testset은 모델 성능 평가에 사용
#이미지를 tensor로 변환하고 픽셀값을 정규화하는 것을 포함하는 두 데이터셋에
#이전에 정의된 변환 파이프라인을 적용
#pytorch의 dataloader 클래스를 사용하여 두 데이터 셋에 대한 데이터 로더 생성
#trainloader 개체에 대해 shuffle 매개 변수를 true로 설정하면 훈련중에(?) 각 배치의 샘플 순서가 섞임
#모델 성능과 안정성을 개선하는데 도움이 됨
#cifar-10 데이터셋에 대한 클래스 목록 정의

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#데이터를 시각화하는데 필요한 라이브러리를 가져옴
#pyplot 모듈 -> 선 도표, 산점도, 히스토그램 및 기타 유형의 도표를 작성하기 위한 도구 제공
#numpy 라이브러리 -> 수치 계산을 위해 배열, 행렬 및 기타 수학적 개체 작업을 위한 도구 제공

In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()
    
#pytorch 텐서(이미지)를 입력으로 사용함
#이미지의 픽셀 값을 정규화 -> [-1,1] 범위의 픽셀 값이 matplotlib에서 이미지를 표시하기 위한
#예상 범위인 [0,1] 범위로 변환됨
#numpy배열을 변환하여 차원을 CHW -> HWC로 재정렬 (C:channel수, H:높이, W:너비)
#show() 함수를 사용하여 플롯 표시

In [ ]:
detaiter = iter(trainloader)
images, labels = next(detaiter)

#iter() : trainloader 데이터 로더에 대한 반복자를 만든 후 next() 함수를 사용해
#반복자에서 첫 번재 데이터 배치를 가져옴

In [ ]:
imshow(torchvision.utils.make_grid(images))
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

#grid() : 이미지의 텐서를 가져와 imshow()를 사용하여 표시할 수 있는 이미지 그리드를 생성하는 함수
#결과 그리드는 이미지 사이에 패딩이 있고 표시 영역에 맞게 크기가 조정됨
#패딩 : convolusion 후 아웃풋 이미지의 크기 유지 및 edge쪽 픽셀 정보를 더 잘 이용하기 위해 사용
#cifar-10 훈련셋의 첫 번째 데이터 배치를 시각적 및 텍스트로 표현
#기계학습모델을 교육하기 전에 데이터가 올바르게 로드되고 처리되고 있는지 확인하는데 사용가능

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

#nn은 신경망 모델을 구성하기 위한 미리 정의된 레이어 및 빌딩 블록 집합을 제공
#일반적으로 복잡한 신경망 아키텍처를 정의하는데 사용
#nn.functional은 사용자 지정 레이어를 정의하는 데 사용할 수 있는 일반적으로 사용되는
#활성화 함수 및 작업에 대한 기능 인터페이스 집합 제공
#학습 가능한 매개변수가 없는 사용자 지정 활성화 함수 및 기타 작업을 정의하는데 사용
#데이터 셋에 대한 훈련을 위한 신경망 아키텍처를 정의하는데 사용

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
#pytorch의 nn.Module 클래스를 사용하여 신경망 아키텍처 정의
#nn.Module - 신경망 모듈, 매개변수를 캡슐화하는 간편한 방법 -> gpu로 이동, 내보내기, 불러오기 등

#self.conv1, self.conv2 : 신경망(Net)은 두 개의 컨볼루션 레이어
#self.fc1, self.fc2 : 두 개의 완전히 연결된 레이어
#self.fc3 : 입력을 매핑하는 최종 출력 레이어로 구성

#init() : 계층을 만들고 초기화 하여 신경망의 아키텍처 정의
#forward() : 입력 데이터가 출력을 생성하기 위해 레이어를 통해 흐르는 방식을 지정하여
#신경망의 순방향 패스를 정의

#self.conv1(첫 번째 컨볼루션 계층): 3개의 입력채널, 6개의 출력 채널 및 5*5의 커널 크기
#MaxPool2d -> 커널 크기가 2*2인 최대 풀링 계층

#self.conv2(두 번째 컨볼루션 계층): 6개의 입력 채널, 16개의 출력 채널 및 5*5의 커널 크기

#self.fc1 및 self.fc2 -> 두번째 풀링 계층의 출력은 평면화되고 각각 120 및 84 출력 기능이 있는
#두 개의 완전히 연결된 계층을 통과

#최종 출력 레이어(self.fc3) -> cifar10 데이터셋의 10개 클래스에 해당하는 10개의 출력기능

#forward : 신경망의 순방향 전달을 구현 -> 입력텐서 x를 가져와 다음 순서로 신경망의 레이어를 통과
#컨볼루션 레이어1 -> 최대 풀링 레이어-> 컨볼루션 레이어2-> 최대 풀링 레이어-> 평탄화 ->
#완전 연결 레이어1-> ReLU 활성화 함수-> 완전히 연결된 레이어2 -> ReLU 활성화 함수 -> 출력 레이어

#최종 레이어의 출력은 순방향 패스의 출력으로 반환됨

In [ ]:
net = Net()

#cifar-10 데이터 셋에 대한 훈련을 위한 신경망 아키텍처를 나타냄
#Net() 인스턴스를 생성하면 init 메서드에 정의된대로 적절한 매개변수를 사용해
#신경망의 계층 초기화 -> 새로 생성된 신경망 훈련 또는 추론에 사용할 수 있음

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#신경망 훈련에 사용할 손실함수와 최적화 알고리즘 정의
#nn.CrossEntropyLoss() -> 예측 레이블과 대상 레이블 간의 엔트로피 손실을 측정하는 기준을 만듬
#이 손실 함수는 다중 클래스 분류 문제에서 일반적으로 사용
#손실함수 : 얼마나 틀리는지를 알게 하는 함수 -> 최소값을 찾아가는 것이 최적화

#optim.SGD() -> 모멘텀이 있는 확률적 경사하강법을 구현하는 옵티마이저 생성
#신경망 매개변수(net.prarmeters()), 학습률(lr=0.001), 모멘텀(momentum=0.9)를 입력으로 사용
#옵티마이저는 학습 중에 신경망의 매개변수를 업데이트 하기 위해
#매개변수에 대한 손실 함수의 기울기를 사용
#옵티마이저 : 손실함수의 최소값을 찾아가는 것을 수행하는 최적화 알고리즘
#확률적 경사하강법: 훈련 셋에서 랜덤하게 하나의 샘플을 선택해 가파른 경사를 조금 내려간 후
#훈련셋에서 랜덤하게 또 다른 샘플을 하나 선택해 경사를 조금 내려간다 -> 전체 샘플을 모두 사용하면
#이 과정을 반복함
#모멘텀: 경사하강법으로 이동할 때 관성을 부여하는 최적화 기법
